In [ ]:
"""
Request handler for HTTP requests with retry logic and error handling.
"""

import asyncio
import time
from typing import Dict, Optional, Any, Union

import aiohttp
import requests
from loguru import logger


class RequestHandler:
    """
    Handles HTTP requests with retry logic and error handling.
    
    This class encapsulates the HTTP request functionality, providing
    both synchronous and asynchronous request methods with retry logic.
    """

    def __init__(
        self,
        base_url: str,
        default_timeout: Optional[int] = 30,
        long_timeout: Optional[int] = 300,
        max_retries: Optional[int] = 5,
        retry_delay: Optional[int] = 5,
        semaphore_limit: Optional[int] = 50,
        verify_ssl: Optional[bool] = False,
    ):
        """
        Initialize the request handler.
        
        Args:
            base_url: Base URL for all requests
            default_timeout: Default timeout for requests in seconds
            long_timeout: Timeout for long-running requests in seconds
            max_retries: Maximum number of retry attempts
            retry_delay: Delay between retries in seconds
            semaphore_limit: Maximum number of concurrent requests
            verify_ssl: Whether to verify SSL certificates
        """
        self.base_url = base_url
        self.default_timeout = default_timeout
        self.long_timeout = long_timeout
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.semaphore_limit = semaphore_limit
        self.verify_ssl = verify_ssl
        
        # Initialize semaphore for async requests
        self._semaphore = asyncio.Semaphore(semaphore_limit)

    def post(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        files: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None,
    ) -> Optional[requests.Response]:
        """
        Perform a POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            files: Optional files to upload
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if all retries failed
        """
        return self._post_with_retry(
            path=path,
            headers=headers,
            payload=payload,
            files=files,
            retries=self.max_retries,
            timeout=timeout or self.long_timeout,
        )

    async def async_post(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        files: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None,
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            files: Optional files to upload
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if all retries failed
        """
        return await self._async_post_with_retry(
            path=path,
            headers=headers,
            payload=payload,
            files=files,
            retries=self.max_retries,
            timeout=timeout or self.long_timeout,
        )

    def get(
        self, 
        path: str, 
        headers: Dict[str, str], 
        timeout: Optional[int] = None
    ) -> Optional[requests.Response]:
        """
        Perform a GET request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            return requests.get(
                url,
                headers=headers,
                verify=self.verify_ssl,
                timeout=timeout or self.default_timeout,
            )
        except Exception as e:
            logger.error(f"GET request to {url} failed: {str(e)}")
            return None

    async def async_get(
        self, 
        path: str, 
        headers: Dict[str, str], 
        timeout: Optional[int] = None
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous GET request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            async with self._semaphore:
                async with aiohttp.ClientSession() as session:
                    return await session.get(
                        url,
                        headers=headers,
                        timeout=timeout or self.default_timeout,
                        ssl=None if not self.verify_ssl else True,
                    )
        except Exception as e:
            logger.error(f"Async GET request to {url} failed: {str(e)}")
            return None

    def delete(
        self, 
        path: str, 
        headers: Dict[str, str], 
        timeout: Optional[int] = None
    ) -> Optional[requests.Response]:
        """
        Perform a DELETE request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            return requests.delete(
                url,
                headers=headers,
                verify=self.verify_ssl,
                timeout=timeout or self.default_timeout,
            )
        except Exception as e:
            logger.error(f"DELETE request to {url} failed: {str(e)}")
            return None

    async def async_delete(
        self, 
        path: str, 
        headers: Dict[str, str], 
        timeout: Optional[int] = None
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous DELETE request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            async with self._semaphore:
                async with aiohttp.ClientSession() as session:
                    return await session.delete(
                        url,
                        headers=headers,
                        timeout=timeout or self.default_timeout,
                        ssl=None if not self.verify_ssl else True,
                    )
        except Exception as e:
            logger.error(f"Async DELETE request to {url} failed: {str(e)}")
            return None

    def put(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None,
    ) -> Optional[requests.Response]:
        """
        Perform a PUT request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            return requests.put(
                url,
                headers=headers,
                json=payload,
                verify=self.verify_ssl,
                timeout=timeout or self.default_timeout,
            )
        except Exception as e:
            logger.error(f"PUT request to {url} failed: {str(e)}")
            return None

    async def async_put(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None,
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous PUT request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            async with self._semaphore:
                async with aiohttp.ClientSession() as session:
                    return await session.put(
                        url,
                        headers=headers,
                        json=payload,
                        timeout=timeout or self.default_timeout,
                        ssl=None if not self.verify_ssl else True,
                    )
        except Exception as e:
            logger.error(f"Async PUT request to {url} failed: {str(e)}")
            return None

    def _post_with_retry(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        files: Optional[Dict[str, Any]] = None,
        retries: Optional[int] = 5,
        timeout: Optional[int] = 300,
    ) -> Optional[requests.Response]:
        """
        Perform an HTTP POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            files: Optional files to upload
            retries: Maximum number of retries
            timeout: Request timeout in seconds
            
        Returns:
            Response object or None if all retries failed
        """
        url = self.base_url + path
        remaining_retries = retries
        
        while remaining_retries > 0:
            try:
                # Handle payload differently if files are provided to avoid conflicts
                if files is not None:
                    response = requests.post(
                        url,
                        headers=headers,
                        data=payload,  # Use data instead of json when files are present
                        files=files,
                        verify=self.verify_ssl,
                        timeout=timeout,
                    )
                else:
                    response = requests.post(
                        url,
                        headers=headers,
                        json=payload,  # Use json for proper serialization
                        verify=self.verify_ssl,
                        timeout=timeout,
                    )
                
                # Check for HTTP errors
                if response.status_code >= 300:
                    detail = f"Error code from API: {response.status_code} {response.text}"
                    logger.error(f"Request failed: {detail}")
                    
                    # For 5xx errors, retry. For 4xx, return the response
                    if response.status_code < 500:
                        return response
                else:
                    # Success
                    return response
                    
            except Exception as e:
                logger.error(f"Exception during request to {url}: {str(e)}")
            
            # If we get here, we need to retry
            remaining_retries -= 1
            if remaining_retries > 0:
                logger.info(
                    f"Retrying {url} in {self.retry_delay} seconds. "
                    f"Attempts remaining: {remaining_retries}"
                )
                time.sleep(self.retry_delay)
        
        # All retries failed
        logger.error(f"Failed to POST to {url} after {retries} attempts")
        return None

    async def _async_post_with_retry(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        files: Optional[Dict[str, Any]] = None,
        retries: Optional[int] = 5,
        timeout: Optional[int] = 300,
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous HTTP POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            files: Optional files to upload
            retries: Maximum number of retries
            timeout: Request timeout in seconds
            
        Returns:
            Response object or None if all retries failed
        """
        url = self.base_url + path
        remaining_retries = retries
        error = None
        
        while remaining_retries > 0:
            try:
                async with self._semaphore:
                    async with aiohttp.ClientSession() as session:
                        # Handle both JSON payload and form data with files
                        if files:
                            # aiohttp handles multipart form data for files
                            form_data = aiohttp.FormData()
                            
                            # Add any payload fields
                            if payload:
                                for key, value in payload.items():
                                    form_data.add_field(key, str(value))
                            
                            # Add files
                            for key, file_info in files.items():
                                form_data.add_field(
                                    key,
                                    file_info[1],  # File content
                                    filename=file_info[0],  # Filename
                                    content_type=file_info[2]  # Content type
                                )
                                
                            response = await session.post(
                                url,
                                headers=headers,
                                data=form_data,
                                timeout=timeout,
                                ssl=None if not self.verify_ssl else True,
                            )
                        else:
                            # Regular JSON payload
                            response = await session.post(
                                url,
                                headers=headers,
                                json=payload,
                                timeout=timeout,
                                ssl=None if not self.verify_ssl else True,
                            )
                        
                        # Check for HTTP errors
                        if response.status >= 300:
                            detail = f"Error code from API: {response.status} {await response.text()}"
                            logger.error(f"Request failed: {detail}")
                            
                            # For 5xx errors, retry. For 4xx, return the response
                            if response.status < 500:
                                return response
                        else:
                            # Success
                            return response
            
            except Exception as e:
                error = e
                logger.error(
                    f"Error occurred calling API {url}. Details: {str(e)}. "
                    f"Current retry attempt# {retries - remaining_retries + 1} of {retries}."
                )
                
            # If we get here, we need to retry
            remaining_retries -= 1
            if remaining_retries > 0:
                logger.info(f"Sleeping for {self.retry_delay} seconds before next retry")
                await asyncio.sleep(self.retry_delay)
        
        # All retries failed
        logger.error(f"Failed to POST to {url} after {retries} attempts")
        if error:
            logger.error(f"Last error: {str(error)}")
            
        return None